# GlobeStay Travel — Data Cleaning and Descriptive Analysis

This notebook focuses on two scopes only:

- Data cleaning: load, standardize, parse dates/weeks, normalize country labels, coerce numerics, basic outlier handling
- Descriptive analysis: summaries by country/channel and time-series visualizations

Inputs
- `data/data_mmm_2020_post.xlsx` (weekly panel for US, UK, Germany)

Outputs
- Cleaned dataset saved to `output/`
- Summary tables and charts saved to `output/`


In [ ]:
# Imports and configuration
import os
import re
from pathlib import Path
from typing import List

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)
sns.set_theme(style="whitegrid")

# Resolve project root (if running inside `notebook/`, step up one level)
PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "data").exists() and PROJECT_ROOT.name.lower() == "notebook":
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_FILENAME_CANDIDATES: List[str] = [
    "data/data_mmm_2020_post.xlsx",
    "data_mmm_2020_post.xlsx",
]
OUTPUTS_DIR = PROJECT_ROOT / "output"
OUTPUTS_DIR.mkdir(exist_ok=True)

# Utility logging
from datetime import datetime

def log(msg: str) -> None:
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{ts}] {msg}")

# Resolve data path
excel_path: Path | None = None
for candidate in DATA_FILENAME_CANDIDATES:
    p = PROJECT_ROOT / candidate
    if p.exists():
        excel_path = p
        break

if excel_path is None:
    # Fallback: search `data/` first, then project root
    data_dir = PROJECT_ROOT / "data"
    xlsx_files = list(data_dir.glob("*.xlsx")) + list(PROJECT_ROOT.glob("*.xlsx"))
    excel_path = xlsx_files[0] if xlsx_files else None

log(f"Detected Excel file: {excel_path}")



[2025-10-29 16:32:14] Detected Excel file: c:\Users\Administrator.等闲的电脑\Desktop\MMA831\data\data_mmm_2020_post.xlsx


In [13]:
# Ensure inline plots in classic Notebook
%matplotlib inline


In [ ]:
# Load Excel: list sheets and preview
if excel_path is None:
    raise FileNotFoundError("No Excel file found. Place the file under data/ (e.g., data/data_mmm_2020_post.xlsx).")

xlsx = pd.ExcelFile(excel_path)
log(f"Sheets found: {xlsx.sheet_names}")

sheets_to_load = xlsx.sheet_names
frames = {}
for sheet in sheets_to_load:
    try:
        df = pd.read_excel(excel_path, sheet_name=sheet, engine="openpyxl")
        frames[sheet] = df
        log(f"Loaded '{sheet}' with shape {df.shape}")
    except Exception as e:
        log(f"WARN: Failed to load sheet '{sheet}': {e}")

# Show quick glimpse of first sheet
first_sheet = sheets_to_load[0] if sheets_to_load else None
if first_sheet:
    display(frames[first_sheet].head(3))



[2025-10-29 16:32:14] Sheets found: ['Sheet1']
[2025-10-29 16:32:14] Loaded 'Sheet1' with shape (591, 67)


,weekstart,country,totbookings,clicks_email,clicks_ppc_brand,clicks_ppc_non_brand,clicks_remarketing,clicks_shop_GoogleHA,clicks_shop_TripAdvisor,clicks_shop_Trivago,clicks_shop_other,cost_email,cost_ppc_brand,cost_ppc_non_brand,cost_remarketing,cost_shop_GoogleHA,cost_shop_TripAdvisor,cost_shop_Trivago,cost_shop_other,display_imps,display_net_spend_eur,olv_imps,olv_net_spend_eur,yt_imps,yt_cost,brandtv_grp,brandtv_net_spend_eur,drtv_grp,drtv_net_spend_eur,ooh_net_spend_eur,radio_net_spend_eur,print_net_spend_eur,cinema_net_spend_eur,meta_comp_grp,ota_comp_grp,value (currency rate),fb_imps,fb_cost,NewYearsDay,MartinL.KingsDay(US-CA),StValentinesDay,PresidentsDay(US-CA),EasterSunday,EasterMonday,LabourDay(DE-NW),MayDay(GB-EN),ChristsAscensionDay(DE-NW),WhitMonday(DE-NW),Remembrance/MemorialDay(US-CA),BankHoliday(GB-EN),IndependenceDay(US-CA),LabourDay(US-CA),GermanUnityDay(DE-NW),ThanksgivingDay(US-CA),ChristmasDay,BoxingDay,sales_Direct,sales_EMK,sales_Interco,sales_ppc_brand,sales_ppc_nonbrand,sales_Retargeting,sales_shop_googleha,sales_shop_other,sales_shop_tripadvisor,sales_shop_trivago,sales_Strat Part
0,2016-01-04,de,398268,474859.0,378640.216496,2.496438e+06,172069.878486,133462.122833,367987.978037,847827.012246,250527.000002,0.0,68560.240287,2.196494e+06,58684.506092,170087.676546,283108.460235,377911.565361,103139.320644,0.0,0.0,0.0,0.0,12.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,926.0,321.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,183169,16703,867,48228,87808,2040,10423,4678,9503,18302,16547
1,2016-01-11,de,404068,478024.0,386722.377913,2.520208e+06,198123.396746,136016.480131,353354.844840,961232.479212,278878.649717,0.0,70502.248076,2.306422e+06,76380.207088,193176.386595,257639.565836,471254.197728,116137.878276,0.0,0.0,0.0,0.0,247.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,706.0,225.0,1.0,57831.0,89.386325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,184480,16278,858,49076,87383,2236,11107,5019,9143,21148,17340
2,2016-01-18,de,388746,482391.0,372694.075197,2.538803e+06,186402.476444,126167.255474,332119.415416,912773.599042,288057.500725,0.0,61647.322276,2.379034e+06,81636.483459,191568.477802,247711.892237,439863.176592,116345.872180,0.0,0.0,0.0,0.0,10249.0,1.459669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,813.0,231.0,1.0,104754.0,231.027642,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,178895,15822,806,45575,85257,2224,10048,4963,8568,19715,16873


In [15]:
# Standardize column names and concatenate sheets if consistent

def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    # lowercase, strip, replace spaces and special chars with underscores
    new_cols = []
    for c in df.columns:
        c2 = str(c).strip().lower()
        c2 = re.sub(r"[^a-z0-9]+", "_", c2)
        c2 = c2.strip("_")
        new_cols.append(c2)
    df.columns = new_cols
    return df

std_frames = {}
for s, d in frames.items():
    std_frames[s] = standardize_columns(d.copy())

# Identify common columns across sheets
sheet_cols = {s: set(df.columns) for s, df in std_frames.items()}
common_cols = set.intersection(*sheet_cols.values()) if std_frames else set()
log(f"Common columns across sheets: {len(common_cols)}")

# If sheets share a schema, concatenate; else keep separate
if len(std_frames) > 0 and len(common_cols) > 3:
    base = pd.concat([df[list(common_cols)].assign(_source_sheet=s) for s, df in std_frames.items()], ignore_index=True)
else:
    # fallback: pick the first standardized sheet
    key = next(iter(std_frames))
    base = std_frames[key].copy()
    base["_source_sheet"] = key

log(f"Base shape after merge/select: {base.shape}")
base.head(3)


[2025-10-29 16:32:14] Common columns across sheets: 67
[2025-10-29 16:32:14] Base shape after merge/select: (591, 68)


,martinl_kingsday_us_ca,clicks_shop_other,germanunityday_de_nw,clicks_remarketing,yt_imps,thanksgivingday_us_ca,cost_shop_googleha,newyearsday,sales_shop_tripadvisor,cost_ppc_brand,sales_ppc_nonbrand,cinema_net_spend_eur,country,weekstart,radio_net_spend_eur,clicks_ppc_non_brand,brandtv_grp,sales_shop_trivago,cost_email,clicks_shop_tripadvisor,ota_comp_grp,whitmonday_de_nw,christmasday,sales_emk,sales_interco,display_net_spend_eur,remembrance_memorialday_us_ca,cost_shop_trivago,cost_shop_tripadvisor,clicks_shop_googleha,labourday_us_ca,ooh_net_spend_eur,brandtv_net_spend_eur,sales_direct,fb_cost,eastersunday,sales_ppc_brand,independenceday_us_ca,value_currency_rate,christsascensionday_de_nw,clicks_email,boxingday,print_net_spend_eur,clicks_ppc_brand,yt_cost,cost_shop_other,clicks_shop_trivago,drtv_net_spend_eur,bankholiday_gb_en,cost_ppc_non_brand,fb_imps,sales_retargeting,display_imps,presidentsday_us_ca,cost_remarketing,mayday_gb_en,labourday_de_nw,olv_imps,drtv_grp,totbookings,olv_net_spend_eur,meta_comp_grp,sales_strat_part,sales_shop_googleha,sales_shop_other,stvalentinesday,eastermonday,_source_sheet
0,0.0,250527.000002,0.0,172069.878486,12.0,0.0,170087.676546,0.0,9503,68560.240287,87808,0.0,de,2016-01-04,0.0,2.496438e+06,0.0,18302,0.0,367987.978037,321.0,0.0,0.0,16703,867,0.0,0.0,377911.565361,283108.460235,133462.122833,0.0,0.0,0.0,183169,0.000000,0.0,48228,0.0,1.0,0.0,474859.0,0.0,0.0,378640.216496,0.000000,103139.320644,847827.012246,0.0,0.0,2.196494e+06,0.0,2040,0.0,0.0,58684.506092,0.0,0.0,0.0,0.0,398268,0.0,926.0,16547,10423,4678,0.0,0.0,Sheet1
1,0.0,278878.649717,0.0,198123.396746,247.0,0.0,193176.386595,0.0,9143,70502.248076,87383,0.0,de,2016-01-11,0.0,2.520208e+06,0.0,21148,0.0,353354.844840,225.0,0.0,0.0,16278,858,0.0,0.0,471254.197728,257639.565836,136016.480131,0.0,0.0,0.0,184480,89.386325,0.0,49076,0.0,1.0,0.0,478024.0,0.0,0.0,386722.377913,0.000000,116137.878276,961232.479212,0.0,0.0,2.306422e+06,57831.0,2236,0.0,0.0,76380.207088,0.0,0.0,0.0,0.0,404068,0.0,706.0,17340,11107,5019,0.0,0.0,Sheet1
2,1.0,288057.500725,0.0,186402.476444,10249.0,0.0,191568.477802,0.0,8568,61647.322276,85257,0.0,de,2016-01-18,0.0,2.538803e+06,0.0,19715,0.0,332119.415416,231.0,0.0,0.0,15822,806,0.0,0.0,439863.176592,247711.892237,126167.255474,0.0,0.0,0.0,178895,231.027642,0.0,45575,0.0,1.0,0.0,482391.0,0.0,0.0,372694.075197,1.459669,116345.872180,912773.599042,0.0,0.0,2.379034e+06,104754.0,2224,0.0,0.0,81636.483459,0.0,0.0,0.0,0.0,388746,0.0,813.0,16873,10048,4963,0.0,0.0,Sheet1


In [ ]:
# Basic schema expectations and soft validation
EXPECTED_COUNTRY_ALIASES = {
    "us": ["us", "usa", "united_states"],
    "uk": ["uk", "gb", "united_kingdom", "great_britain"],
    "de": ["de", "ger", "germany", "deutschland"],
}

# Try to detect columns of interest
col_map = {
    "country": None,
    "date": None,  # weekly date or week ending
    "week": None,  # week number if present
    "bookings": None,  # total bookings outcome
}

for c in base.columns:
    if col_map["country"] is None and re.search(r"country|market|geo", c):
        col_map["country"] = c
    if col_map["date"] is None and re.search(r"date|week_end|week_start|week_ending", c):
        col_map["date"] = c
    if col_map["week"] is None and re.search(r"^week$|week_num|week_number", c):
        col_map["week"] = c
    if col_map["bookings"] is None and re.search(r"bookings|orders|sales_total|total_sales", c):
        col_map["bookings"] = c

log(f"Detected columns: {col_map}")

# Identify potential spend/impression/performance columns
spend_cols = [c for c in base.columns if re.search(r"(^|_)spend($|_)|cost|cpc|media_spend", c)]
impr_cols = [c for c in base.columns if re.search(r"impr|impressions", c)]
performance_sales_cols = [c for c in base.columns if re.search(r"^sales_.*|.*_sales$", c)]

log(f"Spend cols: {len(spend_cols)}, Impr cols: {len(impr_cols)}, Perf sales cols: {len(performance_sales_cols)}")



[2025-10-29 16:32:14] Detected columns: {'country': 'country', 'date': 'weekstart', 'week': None, 'bookings': None}
[2025-10-29 16:32:14] Spend cols: 18, Impr cols: 0, Perf sales cols: 11


In [17]:
# Fallback: create bookings proxy if not detected
if not col_map["bookings"] and performance_sales_cols:
    clean["bookings_proxy"] = clean[performance_sales_cols].sum(axis=1, min_count=1)
    if clean["bookings_proxy"].notna().any():
        col_map["bookings"] = "bookings_proxy"
        log("No explicit 'bookings' detected; using 'bookings_proxy' derived from performance sales.")
    else:
        log("Bookings proxy contained only NaNs; skipping proxy assignment.")


[2025-10-29 16:32:14] No explicit 'bookings' detected; using 'bookings_proxy' derived from performance sales.


In [18]:
# Cleaning helpers: country normalization, date/week parsing, numeric coercion

COUNTRY_NORMALIZATION = {
    **{alias: "US" for alias in EXPECTED_COUNTRY_ALIASES["us"]},
    **{alias: "UK" for alias in EXPECTED_COUNTRY_ALIASES["uk"]},
    **{alias: "DE" for alias in EXPECTED_COUNTRY_ALIASES["de"]},
}


def normalize_country(val) -> str | None:
    if pd.isna(val):
        return None
    s = str(val).strip().lower()
    return COUNTRY_NORMALIZATION.get(s, str(val).strip())


def parse_weeklike_date(series: pd.Series) -> pd.Series:
    # Try parse as datetime first
    parsed = pd.to_datetime(series, errors="coerce")
    if parsed.notna().mean() > 0.5:
        return parsed.dt.to_period("W").dt.to_timestamp("W-SUN")
    # If parse failed, try integer week numbers with an inferred year (fallback: 2014 as start)
    numeric = pd.to_numeric(series, errors="coerce")
    if numeric.notna().mean() > 0.5:
        # Assume week numbers monotonically increase; build fake dates starting from 2014-01-05 (first Sunday)
        start = pd.Timestamp("2014-01-05")
        return numeric.fillna(0).astype(int).apply(lambda w: start + pd.Timedelta(weeks=max(w-1, 0)))
    return pd.NaT


def coerce_numeric(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

# Apply cleaning
clean = base.copy()

if col_map["country"]:
    clean["country_std"] = clean[col_map["country"]].map(normalize_country)
else:
    clean["country_std"] = None

if col_map["date"]:
    clean["week_start"] = parse_weeklike_date(clean[col_map["date"]])
elif col_map["week"]:
    clean["week_start"] = parse_weeklike_date(clean[col_map["week"]])
else:
    clean["week_start"] = pd.NaT

numeric_candidates = []
numeric_candidates += spend_cols
numeric_candidates += impr_cols
numeric_candidates += performance_sales_cols
if col_map["bookings"]:
    numeric_candidates.append(col_map["bookings"])

numeric_candidates = sorted(set(numeric_candidates))
clean = coerce_numeric(clean, numeric_candidates)

# Simple outlier capping per column (95th percentile), only for spend/impressions
for c in spend_cols + impr_cols:
    if c in clean.columns:
        q = clean[c].quantile(0.995)
        if pd.notna(q) and q > 0:
            clean[c] = np.clip(clean[c], a_min=None, a_max=q)

# Drop obvious empty rows
before = len(clean)
clean = clean[clean["week_start"].notna() | clean["country_std"].notna()]
after = len(clean)
log(f"Dropped empty rows: {before - after}")

clean.head(3)


[2025-10-29 16:32:14] Dropped empty rows: 15


,martinl_kingsday_us_ca,clicks_shop_other,germanunityday_de_nw,clicks_remarketing,yt_imps,thanksgivingday_us_ca,cost_shop_googleha,newyearsday,sales_shop_tripadvisor,cost_ppc_brand,sales_ppc_nonbrand,cinema_net_spend_eur,country,weekstart,radio_net_spend_eur,clicks_ppc_non_brand,brandtv_grp,sales_shop_trivago,cost_email,clicks_shop_tripadvisor,ota_comp_grp,whitmonday_de_nw,christmasday,sales_emk,sales_interco,display_net_spend_eur,remembrance_memorialday_us_ca,cost_shop_trivago,cost_shop_tripadvisor,clicks_shop_googleha,labourday_us_ca,ooh_net_spend_eur,brandtv_net_spend_eur,sales_direct,fb_cost,eastersunday,sales_ppc_brand,independenceday_us_ca,value_currency_rate,christsascensionday_de_nw,clicks_email,boxingday,print_net_spend_eur,clicks_ppc_brand,yt_cost,cost_shop_other,clicks_shop_trivago,drtv_net_spend_eur,bankholiday_gb_en,cost_ppc_non_brand,fb_imps,sales_retargeting,display_imps,presidentsday_us_ca,cost_remarketing,mayday_gb_en,labourday_de_nw,olv_imps,drtv_grp,totbookings,olv_net_spend_eur,meta_comp_grp,sales_strat_part,sales_shop_googleha,sales_shop_other,stvalentinesday,eastermonday,_source_sheet,country_std,week_start
0,0.0,250527.000002,0.0,172069.878486,12.0,0.0,170087.676546,0.0,9503.0,68560.240287,87808.0,0.0,de,2016-01-04,0.0,2.496438e+06,0.0,18302.0,0.0,367987.978037,321.0,0.0,0.0,16703.0,867.0,0.0,0.0,377911.565361,283108.460235,133462.122833,0.0,0.0,0.0,183169.0,0.000000,0.0,48228.0,0.0,1.0,0.0,474859.0,0.0,0.0,378640.216496,0.000000,103139.320644,847827.012246,0.0,0.0,2.196494e+06,0.0,2040.0,0.0,0.0,58684.506092,0.0,0.0,0.0,0.0,398268,0.0,926.0,16547.0,10423.0,4678.0,0.0,0.0,Sheet1,DE,2016-01-10
1,0.0,278878.649717,0.0,198123.396746,247.0,0.0,193176.386595,0.0,9143.0,70502.248076,87383.0,0.0,de,2016-01-11,0.0,2.520208e+06,0.0,21148.0,0.0,353354.844840,225.0,0.0,0.0,16278.0,858.0,0.0,0.0,471254.197728,257639.565836,136016.480131,0.0,0.0,0.0,184480.0,89.386325,0.0,49076.0,0.0,1.0,0.0,478024.0,0.0,0.0,386722.377913,0.000000,116137.878276,961232.479212,0.0,0.0,2.306422e+06,57831.0,2236.0,0.0,0.0,76380.207088,0.0,0.0,0.0,0.0,404068,0.0,706.0,17340.0,11107.0,5019.0,0.0,0.0,Sheet1,DE,2016-01-17
2,1.0,288057.500725,0.0,186402.476444,10249.0,0.0,191568.477802,0.0,8568.0,61647.322276,85257.0,0.0,de,2016-01-18,0.0,2.538803e+06,0.0,19715.0,0.0,332119.415416,231.0,0.0,0.0,15822.0,806.0,0.0,0.0,439863.176592,247711.892237,126167.255474,0.0,0.0,0.0,178895.0,231.027642,0.0,45575.0,0.0,1.0,0.0,482391.0,0.0,0.0,372694.075197,1.459669,116345.872180,912773.599042,0.0,0.0,2.379034e+06,104754.0,2224.0,0.0,0.0,81636.483459,0.0,0.0,0.0,0.0,388746,0.0,813.0,16873.0,10048.0,4963.0,0.0,0.0,Sheet1,DE,2016-01-24


In [19]:
# Save cleaned dataset
clean_out_csv = OUTPUTS_DIR / "cleaned_globestay.csv"
clean_out_parquet = OUTPUTS_DIR / "cleaned_globestay.parquet"

clean.to_csv(clean_out_csv, index=False)
try:
    clean.to_parquet(clean_out_parquet, index=False)
except Exception as e:
    log(f"WARN: Parquet save failed: {e}")

log(f"Saved cleaned CSV: {clean_out_csv}")
log(f"Saved cleaned Parquet (if succeeded): {clean_out_parquet}")


[2025-10-29 16:32:14] Saved cleaned CSV: c:\Users\Administrator.等闲的电脑\Desktop\MMA831\output\cleaned_globestay.csv
[2025-10-29 16:32:14] Saved cleaned Parquet (if succeeded): c:\Users\Administrator.等闲的电脑\Desktop\MMA831\output\cleaned_globestay.parquet


In [20]:
# Descriptive analytics — basic summaries

summary_blocks = {}

# Coverage
summary_blocks["row_counts_by_country"] = (
    clean.assign(country_std=clean["country_std"].fillna("Unknown"))
         .groupby("country_std", dropna=False)
         .size()
         .rename("rows")
         .reset_index()
)

# Time coverage
if "week_start" in clean.columns and clean["week_start"].notna().any():
    summary_blocks["time_coverage"] = pd.DataFrame({
        "min_week": [clean["week_start"].min()],
        "max_week": [clean["week_start"].max()],
        "num_weeks": [clean["week_start"].nunique()],
    })

# Spend and impressions by country
if spend_cols:
    summary_blocks["spend_by_country"] = (
        clean.groupby("country_std")[spend_cols].sum(min_count=1).reset_index()
    )
if impr_cols:
    summary_blocks["impr_by_country"] = (
        clean.groupby("country_std")[impr_cols].sum(min_count=1).reset_index()
    )

# Bookings by country
if col_map["bookings"]:
    summary_blocks["bookings_by_country"] = (
        clean.groupby("country_std")[col_map["bookings"]].sum(min_count=1).reset_index()
    )

# Show summaries
for name, df in summary_blocks.items():
    log(f"Summary: {name}")
    display(df.head(10))


KeyError: 'Column not found: bookings_proxy'

In [ ]:

if col_map["bookings"] and clean["week_start"].notna().any():
    plt.figure(figsize=(11, 5))
    ts = clean.dropna(subset=["week_start"]).copy()
    ts = ts.sort_values("week_start")
    sns.lineplot(
        data=ts, x="week_start", y=col_map["bookings"], hue="country_std", estimator=None
    )
    plt.title("Weekly bookings over time by country")
    plt.xlabel("Week start")
    plt.ylabel("Bookings")
    plt.tight_layout()
    fig_path = OUTPUTS_DIR / "ts_bookings_by_country.png"
    plt.savefig(fig_path, dpi=150)
    plt.show()
    log(f"Saved chart: {fig_path}")

if spend_cols and clean["week_start"].notna().any():
    ts = clean.dropna(subset=["week_start"]).copy()
    ts["total_spend"] = ts[spend_cols].sum(axis=1, min_count=1)
    plt.figure(figsize=(11, 5))
    sns.lineplot(data=ts.sort_values("week_start"), x="week_start", y="total_spend", hue="country_std", estimator=None)
    plt.title("Weekly total media spend over time by country")
    plt.xlabel("Week start")
    plt.ylabel("Total spend")
    plt.tight_layout()
    fig_path = OUTPUTS_DIR / "ts_total_spend_by_country.png"
    plt.savefig(fig_path, dpi=150)
    plt.show()
    log(f"Saved chart: {fig_path}")



In [ ]:

value_cols = [c for c in clean.columns if re.match(r"(spend|sales|impr)[_].+", c)]

if value_cols:
    tidy = (
        clean[["week_start", "country_std"] + value_cols]
        .melt(id_vars=["week_start", "country_std"], var_name="metric_channel", value_name="value")
    )
    parts = tidy["metric_channel"].str.split("_", n=1, expand=True)
    tidy["metric"] = parts[0]
    tidy["channel"] = parts[1]

    # Summaries by country-channel
    channel_summary = (
        tidy.groupby(["country_std", "channel", "metric"]) ["value"].sum(min_count=1).reset_index()
            .pivot(index=["country_std", "channel"], columns="metric", values="value")
            .reset_index()
    )
    display(channel_summary.head(20))

    # Save
    channel_summary_out = OUTPUTS_DIR / "channel_summary.csv"
    channel_summary.to_csv(channel_summary_out, index=False)
    log(f"Saved channel summary: {channel_summary_out}")
else:
    log("No channel-like columns detected (spend_*, sales_*, impr_*). Skipping channel summary.")


metric,country_std,channel,sales
0,1000088.88063893,direct,408697.0
1,1000088.88063893,emk,30191.0
2,1000088.88063893,interco,76557.0
3,1000088.88063893,ppc_brand,37267.0
4,1000088.88063893,ppc_nonbrand,191430.0
5,1000088.88063893,retargeting,14774.0
6,1000088.88063893,shop_googleha,85141.0
7,1000088.88063893,shop_other,18107.0
8,1000088.88063893,shop_tripadvisor,50203.0
9,1000088.88063893,shop_trivago,57537.0


[2025-10-29 16:26:35] Saved channel summary: c:\Users\Administrator.等闲的电脑\Desktop\MMA831\output\channel_summary.csv
